# Refactor bus corridors

In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(900_000_000_000) ## 800GB?

import datetime as dt
import geopandas as gpd
import pandas as pd
import zlib

from siuba import *

import utilities
import A1_rail_ferry_brt as rail_ferry_brt

from shared_utils import rt_utils

GCS_FILE_PATH = utilities.GCS_FILE_PATH

analysis_date = rail_ferry_brt.analysis_date
analysis_date

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


datetime.date(2022, 6, 15)

In [ ]:
# calitp-analytics-data/data-analyses/high_quality_transit_areas/bus_corridors/182_bus.parquet
bus_corridor = gpd.read_parquet(f"{GCS_FILE_PATH}bus_corridors/182_bus.parquet")

In [ ]:
#bus_corridor2 = gpd.read_parquet("./data/182_bus2.parquet")

In [2]:
itp_id = 182
date_str = analysis_date.strftime(rt_utils.FULL_DATE_FMT)

# Skip writing geoparquet again for now 
# TODO: tweak rt_utils to overwrite export? 
# Overwriting while testing this is not ideal, don't want to mess it up

#routelines = rt_utils.get_routelines(itp_id, analysis_date)
#routelines = gpd.read_parquet(f"{rt_utils.GCS_FILE_PATH}"
#                             f"cached_views/routelines_{itp_id}_{date_str}.parquet" 
#                            )

#routelines.to_parquet(f"./data/routelines_{itp_id}_{date_str}.parquet")

## force clear to ensure route type data present
#trips = rt_utils.get_trips(itp_id, analysis_date, force_clear=True, route_types = ['3'])
#trips.to_parquet(f"./data/trips_{itp_id}_{date_str}.parquet")

#stop_times = rt_utils.get_stop_times(itp_id, analysis_date)
#stop_times.to_parquet(f"./data/st_{itp_id}_{date_str}.parquet")

#stops = rt_utils.get_stops(itp_id, analysis_date)
#stops.to_parquet(f"./data/stops_{itp_id}_{date_str}.parquet")

In [ ]:
routelines = gpd.read_parquet(f"./data/routelines_{itp_id}_{date_str}.parquet")
trips = pd.read_parquet(f"./data/trips_{itp_id}_{date_str}.parquet")
stop_times = pd.read_parquet(f"./data/st_{itp_id}_{date_str}.parquet")
stops = gpd.read_parquet(f"./data/stops_{itp_id}_{date_str}.parquet")

In [ ]:
#!pip install dask

In [3]:
import dask
import dask.dataframe as dd


ddf = dd.read_parquet(f"{rt_utils.GCS_FILE_PATH}"
                             f"cached_views/st_{itp_id}_{date_str}.parquet")
stop_times = pd.read_parquet(f"{rt_utils.GCS_FILE_PATH}"
                             f"cached_views/st_{itp_id}_{date_str}.parquet")

In [4]:
ddf = dd.read_parquet(f"{rt_utils.GCS_FILE_PATH}"
                             f"cached_views/st_{itp_id}_{date_str}.parquet")

#https://stackoverflow.com/questions/45428292/how-to-convert-pandas-str-split-call-to-to-dask
ddf = ddf.assign(
    **{"departure_hour": ddf.departure_time.str.partition(":")[0].astype(int)}
)

#https://stackoverflow.com/questions/54955833/apply-a-lambda-function-to-a-dask-dataframe
ddf["departure_hour"] = ddf.departure_hour.map(lambda x: x-24 if x >=24 else x)

In [5]:
trips_per_hour = (ddf.groupby(["calitp_itp_id", "stop_id", "departure_hour"])
                  .agg({'trip_id': 'count'})
                  .reset_index()
                  .rename(columns = {"trip_id": "n_trips"})
                 )

In [6]:
type(trips_per_hour)

dask.dataframe.core.DataFrame

## All shapes for operator
* Dissolve by length, and generate segments at once
* Join stops once

In [ ]:
stop_times.columns